In [3]:
from gensim import models

ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin')

print(ko_model.wv["민법"])
print("신의성실의" in ko_model.wv.key_to_index)

[ 1.31596327e-01  2.22926497e-01  3.86130475e-02  7.84323066e-02
 -5.16258404e-02  7.97712430e-03  1.99839935e-01 -5.34022510e-01
  4.55676764e-01 -1.09818868e-01  2.69600868e-01  1.33270577e-01
 -7.08474442e-02  3.92643005e-01 -8.30028057e-02  1.95343614e-01
  1.23976536e-01  7.39005357e-02 -5.55521846e-01  4.08919267e-02
 -4.10625562e-02  4.59080249e-01  1.87489107e-01  2.35088855e-01
  2.62020618e-01  1.62249303e+00  4.06613946e-02  1.77449897e-01
  7.33493194e-02  1.39033973e-01  8.16323459e-02 -4.35715675e-01
 -3.06389809e-01  6.05222285e-01 -1.34828299e-01 -1.07189439e-01
 -4.52251770e-02 -3.82874668e-01  2.79095650e-01 -4.98226881e-02
 -3.65950465e-01  3.71522725e-01 -1.66791379e-01  5.06538570e-01
  2.26466656e-01  1.24222279e-01 -3.29030573e-01  5.58425188e-02
  6.66163191e-02  1.33904546e-01 -3.03513199e-01 -3.30507636e-01
  1.03688389e-01 -2.50025034e-01  1.09766692e-01 -3.00087482e-01
 -1.08001627e-01 -4.46593836e-02 -3.73714268e-01 -4.91346195e-02
 -1.81760564e-01  7.92532

In [44]:
import requests

def getWikiData(search):
    
    url = "https://ko.wikipedia.org/w/api.php?action=parse&parse&page="+ search +"&prop=wikitext&formatversion=2&format=json"
    headers = {'Content-Type': 'application/json'}

    response = requests.get(url, headers=headers)

    return response

origintext = getWikiData("민법").json()

# print(text)

# for key, value in text['parse'].items():
#     print(f'{key}: {value}\n\n')
    
# print(text['parse']['wikitext'])

# from icu_tokenizer import Tokenizer
from kiwipiepy import Kiwi

kiwi = Kiwi()
kiwi.prepare()


text_wikitext = text['parse']['wikitext']
result = kiwi.tokenize(text_wikitext)
print(result)

# from sklearn.metrics.pairwise import cosine_similarity

# for token in result:
#     if token in ko_model.wv.key_to_index:
#         cosine_sim = cosine_similarity([ko_model.wv[text['parse']['title']]], [ko_model.wv[token]])
#         if cosine_sim > 0.5:
#             print(f'{token}: {cosine_sim}')
#     # else: 
#     #     print(token)

# from gensim import models

# ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin')

# print(ko_model.wv["민법"])
# print("신의성실의" in ko_model.wv.key_to_index)


[Token(form='{', tag='SSO', start=0, len=1), Token(form='{', tag='SSO', start=1, len=1), Token(form='위키데이터', tag='NNP', start=2, len=5), Token(form='속성', tag='NNG', start=8, len=2), Token(form='추적', tag='NNG', start=11, len=2), Token(form='}', tag='SSC', start=13, len=1), Token(form='}', tag='SSC', start=14, len=1), Token(form="'", tag='SSC', start=16, len=1), Token(form="'", tag='SSC', start=17, len=1), Token(form="'", tag='SSO', start=18, len=1), Token(form='민법', tag='NNG', start=19, len=2), Token(form="'", tag='SSC', start=21, len=1), Token(form="'", tag='SSO', start=22, len=1), Token(form="'", tag='SSC', start=23, len=1), Token(form='(', tag='SSO', start=24, len=1), Token(form='民法', tag='SH', start=25, len=2), Token(form=',', tag='SP', start=27, len=1), Token(form='{', tag='SSO', start=29, len=1), Token(form='{', tag='SSO', start=30, len=1), Token(form='llang', tag='SL', start=31, len=5), Token(form='|', tag='SW', start=36, len=1), Token(form='fr', tag='SL', start=37, len=2), Token

In [48]:
cosine_list = []
made_words = []
temp_word = ""
bef_tag = ""
bef_loc = 0
form_num = 0
cnt = 0
cosine = {}
cosine_sum = 0
from sklearn.metrics.pairwise import cosine_similarity
for form, tag, start, len in result:

  # 먼저 명사일 때
  if tag in ['NNP', 'NNG']:

    # 명사 여러 개가 띄어쓰기 없이 나왔을 때 한 단어로 취급, 후에 바꿔도 됨(모든 명사 구분하는 걸로)
    if bef_tag == 'NN' and bef_loc == start:
      temp_word += form
      
    # 띄어쓰기 후에 들어오는 명사
    elif bef_tag == 'NN' and bef_loc != start:
      made_words.append(temp_word)
      temp_word = form
      form_num = 0

    # 관형격 조사 다음에 들어오는 명사
    elif bef_tag == 'JKG' or bef_tag == 'XSN':
      cosine_list.append(form)
      temp_word += form
      
    # 처음 들어오는 명사
    elif temp_word == "":
      temp_word = form

    # 단어 위치, 단어 개수 초기화
    bef_loc = start + len
    form_num += 1
    bef_tag = 'NN'
    cnt += 1

  # 관형격 조사일 때 ex)신의성실의 원칙
  elif tag == 'JKG' and bef_tag == "NN":
    cosine_list.append(form)
    temp_word += form + " "
    bef_tag = 'JKG'
    form_num += 1

  # ~적
  elif tag == 'XSN' and bef_tag == "NN":
    temp_word += form + " "
    bef_tag = 'XSN'
    form_num += 1


  # 다른 태그 들어왔을때 단어 리스트에 넣기 및 초기화
  # 더 고려할 것 : 몇 개의 형태소로 이루어진 것만 넣을까, 1글자거나 긴 단어는 뺄까, 1 <= form_num <= 4 and ~의 경우 빼기
  else:
    # if temp_word not in made_words:
    del len
    if bef_tag == 'NN' and len(temp_word) > 1:
      made_words.append(temp_word)
      for token in cosine_list:
        if token in ko_model.wv.key_to_index:
            cosine_sim = cosine_similarity([ko_model.wv[text['parse']['title']]], [ko_model.wv[token]])
            if cosine_sum < cosine_sim:
                cosine_sum = cosine_sim
      cosine[temp_word] = cosine_sum
      # print(temp_word)
    cosine_list = []
    cosine_sum = 0
    temp_word = ""
    bef_tag = ""
    bef_loc = 0
    form_num = 0

    
def count_words(text):
    word_counts = 0
    for form, tag, start, len in result:
        if text==form:
            word_counts+=1
    
    return word_counts


In [49]:
made_words_set = set(made_words)
final_words_list = list(made_words_set)


for token in final_words_list:
    if token in ko_model.wv.key_to_index:
        cosine_sim = cosine_similarity([ko_model.wv[text['parse']['title']]], [ko_model.wv[token]])
        if cosine_sim > 0.3:
            count = count_words(token)
            print(f'{token}: {cosine_sim} : {count}')
    # else: 
    #     print(token)
for token in cosine:
    if cosine[token] > 0.1:
        count = count_words(token)
        print(f'{token} : {cosine[token]} : {count}')

무과실책임: [[0.30464822]] : 1
질권: [[0.39972463]] : 1
일본: [[0.32041776]] : 2
법: [[0.35164395]] : 69
부칙: [[0.40598136]] : 1
독일: [[0.37291718]] : 3
계약법: [[0.4910602]] : 41
성문법: [[0.32886437]] : 1
물권: [[0.53472865]] : 5
물권법: [[0.5046581]] : 10
지역권: [[0.3398642]] : 1
임의규정: [[0.34756717]] : 2
임의법: [[0.40234265]] : 0
개정: [[0.3433419]] : 6
총칙: [[0.5663412]] : 3
채권: [[0.3861485]] : 20
전형계약: [[0.38813642]] : 1
상법: [[0.58560395]] : 6
법률: [[0.37267113]] : 10
통설: [[0.4097653]] : 1
공법: [[0.4022437]] : 1
법률행위: [[0.40402576]] : 3
지상권: [[0.3872638]] : 1
사법인: [[0.30098438]] : 0
민법: [[0.99999994]] : 44
재산관계: [[0.3100411]] : 0
판덱텐: [[0.3174858]] : 3
채권법: [[0.50105566]] : 8
현행: [[0.38005242]] : 3
일반법: [[0.39126202]] : 0
조문: [[0.33936274]] : 1
본문: [[0.30383974]] : 7
재산법: [[0.41880167]] : 0
규정: [[0.31683043]] : 19
상속법: [[0.47404587]] : 0
영미법: [[0.37272617]] : 1
전세권: [[0.33863658]] : 1
권리: [[0.3078683]] : 6
소멸시효: [[0.31691015]] : 1
민사법: [[0.5046778]] : 5
대한민국: [[0.31067786]] : 5
미국: [[0.30622253]] : 1
관습법: [[0.40